In [ ]:
%pip install openai
%pip install pandas
%pip install numpy
%pip install nltk
%pip install gensim

In [ ]:
#Azure OpenAI Setup

import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key= "28f146892bb54fd3a8ed90c8f7080f0a",
    api_version="2023-12-01-preview",
    azure_endpoint= "https://assignments.openai.azure.com/"
)

In [1]:
#Pre process the text data
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
import numpy

from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup


REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "html.parser").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [4]:
#Code to generate embeddings and store in a collection in VectorDB

import json
import time

import smartassignments
import vectordb

client = smartassignments.GetOpenAiClient()

collection = vectordb.CreateCollection("smartassignments")

def ReadAndGenerateEmbeddings(client, fileName):
    #read from text file
    with open(fileName, 'r') as file: 
        data = file.read()

        #data is a json
        data = json.loads(data)

        for key in data.keys():
            curr_data = data[key]

            #data is role and work items
            name = curr_data['Name']
            role = curr_data['Role']
            workItems = curr_data['Tasks']

            meta_data = {
                "Name": name,
                "Role": role
            }

            #create a json using both
            curr_data = {
                "role": role,
                "tasks": [clean_text(task) for task in workItems]
            }

            #convert to string
            curr_data = json.dumps(curr_data)

            #get embeddings for the jso
            embedding = smartassignments.GetEmbeddings(client,  curr_data)[0].tolist()

            #also get the summary of the embeddings
            summary = smartassignments.GetSummary(client, curr_data)

            #store the embedding and summary in the collection
            vectordb.Add(collection, key, embedding,  meta_data, summary)

            #verify if the data is stored
            query_embedding = vectordb.Get(collection, key)

            #wait for some time
            time.sleep(6)

    return


ReadAndGenerateEmbeddings(client, "data.json")


C:\Users\chaparasagar\AppData\Local\Temp\ipykernel_14764\2332485162.py:25: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").text # HTML decoding
